# StableDiffusion

In [ ]:
# Instala as dependências necessárias
#!pip install torch transformers diffusers gradio accelerate 

In [ ]:
# Importa as bibliotecas necessárias
from diffusers import StableDiffusionPipeline  # Biblioteca para o pipeline da Stable Diffusion
import torch  # Biblioteca para operações com tensores
import time  # Biblioteca para funções relacionadas ao tempo


In [ ]:
# Define o ID do modelo
model_id = "CompVis/stable-diffusion-v1-4"
# Carrega o modelo pré-treinado da Stable Diffusion com configuração para uso eficiente de memória
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, low_cpu_mem_usage=True)
pipe = pipe.to("cuda")   # Move o modelo para a GPU (se disponível) para melhorar o desempenho

In [ ]:
# Função para gerar imagens com parâmetros ajustáveis
def generate_image(prompt, style=None, width=512, height=512, num_inference_steps=50, guidance_scale=7.5, seed=None):
    if style:  # Se um estilo for fornecido, adiciona-o ao prompt
        prompt = f"{prompt}, {style}"
    
    # Configura a semente aleatória, se fornecida
    if seed is not None:
        generator = torch.manual_seed(seed)  # Define a semente aleatória
    else:
        generator = None
    
   
    # Gera a imagem com os parâmetros fornecidos
    image = pipe(
        prompt, 
        width=width, 
        height=height, 
        num_inference_steps=num_inference_steps, 
        guidance_scale=guidance_scale, 
        generator=generator
    ).images[0]
    
    return image  # Retorna a imagem gerada

In [ ]:
# Lista de estilos sugeridos
styles = [
    "cyberpunk style",
    "impressionist style",
    "watercolor",
    "pixel art",
    "realistic",
    "abstract",
    "surreal",
    "steampunk",
    "modern art",
    "baroque",
    "and more"
]

In [ ]:
# Importa a biblioteca Gradio para criar a interface web
import gradio as gr

# Função que será chamada pela interface Gradio
def generate_image_interface(prompt, style, width, height, num_inference_steps, guidance_scale, seed):
    image = generate_image(prompt, style, width, height, num_inference_steps, guidance_scale, seed)  # Gera a imagem
    return image  # Retorna a imagem gerada para a interface

# Define a interface Gradio
iface = gr.Interface(
    fn=generate_image_interface,  # Função a ser chamada pela interface
    inputs=[
        gr.Textbox(label="Prompt"),  # Caixa de texto para o prompt
        gr.Textbox(label="Style"),  # Caixa de texto para o estilo
        gr.Slider(256, 1024, step=64, label="Width", value=512),  # Slider para a largura da imagem
        gr.Slider(256, 1024, step=64, label="Height", value=512),  # Slider para a altura da imagem
        gr.Slider(1, 100, step=1, label="Number of Inference Steps", value=50),  # Slider para o número de passos de inferência
        gr.Slider(0.1, 20.0, step=0.1, label="Guidance Scale", value=7.5),  # Slider para a escala de orientação
        gr.Number(label="Seed", value=None)  # Caixa de número para a semente aleatória
    ],
    outputs=gr.Image(type="pil"),  # Define a saída como uma imagem do tipo PIL
    title="Image Generator",  # Título da interface
    description="Enter a prompt and optionally a style. Available styles: " + ", ".join(styles)  # Descrição da interface com os estilos disponíveis
)

In [ ]:
# Lança a interface Gradio
iface.launch()